Introduction: 
==========
The basic idea behind this project is to cluster the neighborhoods in Toronto to get the best places to open a restaurant, a coffee shop, or a hotel. 

In [1]:
#Importing needed Libraries 
import pandas as pd 
import folium 
from sklearn.cluster import KMeans 
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
import requests

The data used here is the same as in week 3 

In [2]:
toronto=pd.read_csv('Toronto_upgraded.csv')
toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188


In [3]:
#latitude and longitude of Toronto
latitude=43.6532
longitude=-79.3832
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [5]:
i=toronto['Borough'].unique()
y={}
for s,x in enumerate (i) :
    y[x]=s
#Enumerating the boroughs to get a different color for each borough on the map    
num_bor=10        
x = np.arange(num_bor)
ys = [i + x + (i*x)**2 for i in range(num_bor)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
latitude=43.6532
longitude=-79.3832

toronto_bor=folium.Map(location=[latitude,longitude],zoom_start=10)
# add markers to the map
markers_colors = []
for lat, lon, poi, bor in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighborhood'], toronto['Borough']):
    label = folium.Popup(str(poi) + ' ,' + str(bor), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[y[bor]-1],
        fill=True,
        fill_color=rainbow[y[bor]-1],
        fill_opacity=0.7).add_to(toronto_bor)
toronto_bor

In [16]:
#Saving the map for the presentation
map_toronto.save("Toronto_map.html")
toronto_bor.save("colored_boroughs.html")

Methodolgy
=========
I'll first use the foursquare to get the and clean the data I need about venues then I'll start using the K-means algorithm

Foursquare
-----------------

In [6]:
#setting the foursquare info
#The API credentials
CLIENT_ID = 'AGUKOD2LWDFUIKRMR2ENS0PYT2IMLUWS5S2FE45QCZIVTLQG'
CLIENT_SECRET = 'BSTVJPKKQO5OSOMBHV4FF3TT5JICNNTTWQRNP1YMLU2Q5NHM' 
VERSION = '20180605' 
radius=500
LIMIT=100
#These IDs were used to return a specific venue only 
food_id='4d4b7105d754a06374d81259'
coffee_id='4bf58dd8d48988d1e0931735'
hotel_id='4bf58dd8d48988d1fa931735'

In [11]:
def getNearbyPlaces(names, latitudes, longitudes, category,unwanted_data=unwanted_categories, radius=500):
    #A function that returens a specific venue based on the category 
    
    venues_list=[]
    i=0
    for name, lat, lng in zip(names, latitudes, longitudes):
            print(i,name)
            i+=1
        # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
                CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, category, radius, LIMIT)
            
                            
            
        # make the GET request
        #try-except statement for when errors occur the rest of the values will be filled with none
            try:
                results = requests.get(url).json()["response"]['groups'][0]['items']
                venues_list.append([(
                            name, 
                            lat, 
                            lng, 
                            v['venue']['categories'][0]['name'], 
                            v['venue']['name'], 
                            v['venue']['location']['lat'], 
                            v['venue']['location']['lng']) for v in results])
            except : 
                   venues_list.append([(
                        name,
                        lat,
                        lng,
                        'None', 
                        'None', 
                        'None',
                        'None')])
            nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
            nearby_venues.columns = ['Neighborhood', 
                      'Neighborhood Latitude', 
                      'Neighborhood Longitude', 
                        'Venue Category',          
                      'Venue', 
                      'Venue Latitude', 
                      'Venue Longitude' ]
    
    return(nearby_venues)

In [17]:
#I had to increase the radius for coffes because at first the api found nothing 
coffees=getNearbyPlaces(names=toronto['Neighborhood'],
                                            latitudes=toronto['Latitude'],
                                            longitudes=toronto['Longitude'] ,
                                            category=coffee_id,
                                            radius=1000)

coffees.head()

0 Parkwoods
1 Victoria Village
2 Regent Park, Harbourfront
3 Lawrence Manor, Lawrence Heights
4 Queen's Park, Ontario Provincial Government
5 Islington Avenue, Humber Valley Village
6 Malvern, Rouge
7 Don Mills
8 Parkview Hill, Woodbine Gardens
9 Garden District, Ryerson
10 Glencairn
11 West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
12 Rouge Hill, Port Union, Highland Creek
13 Don Mills
14 Woodbine Heights
15 St. James Town
16 Humewood-Cedarvale
17 Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
18 Guildwood, Morningside, West Hill
19 The Beaches
20 Berczy Park
21 Caledonia-Fairbanks
22 Woburn
23 Leaside
24 Central Bay Street
25 Christie
26 Cedarbrae
27 Hillcrest Village
28 Bathurst Manor, Wilson Heights, Downsview North
29 Thorncliffe Park
30 Richmond, Adelaide, King
31 Dufferin, Dovercourt Village
32 Scarborough Village
33 Fairview, Henry Farm, Oriole
34 Northwood Park, York University
35 East Toronto, Broadview North (Old East York)
36 Harbourfr

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Category,Venue,Venue Latitude,Venue Longitude
0,Parkwoods,43.75245,-79.32991,Café,Tim Hortons,43.760668,-79.326368
1,Parkwoods,43.75245,-79.32991,Coffee Shop,Tim Hortons,43.760670,-79.326589
2,Parkwoods,43.75245,-79.32991,Coffee Shop,La Notre,43.760704,-79.325396
3,Victoria Village,43.73057,-79.31306,Coffee Shop,Tim Hortons,43.725517,-79.313103
4,"Regent Park, Harbourfront",43.65512,-79.36264,Coffee Shop,Tandem Coffee,43.653559,-79.361809


In [41]:
restaurants=getNearbyPlaces(names=toronto['Neighborhood'],
                                            latitudes=toronto['Latitude'],
                                            longitudes=toronto['Longitude'] ,
                                            category=food_id )
restaurants.head()

0 Parkwoods
1 Victoria Village
2 Regent Park, Harbourfront
3 Lawrence Manor, Lawrence Heights
4 Queen's Park, Ontario Provincial Government
5 Islington Avenue, Humber Valley Village
6 Malvern, Rouge
7 Don Mills
8 Parkview Hill, Woodbine Gardens
9 Garden District, Ryerson
10 Glencairn
11 West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
12 Rouge Hill, Port Union, Highland Creek
13 Don Mills
14 Woodbine Heights
15 St. James Town
16 Humewood-Cedarvale
17 Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
18 Guildwood, Morningside, West Hill
19 The Beaches
20 Berczy Park
21 Caledonia-Fairbanks
22 Woburn
23 Leaside
24 Central Bay Street
25 Christie
26 Cedarbrae
27 Hillcrest Village
28 Bathurst Manor, Wilson Heights, Downsview North
29 Thorncliffe Park
30 Richmond, Adelaide, King
31 Dufferin, Dovercourt Village
32 Scarborough Village
33 Fairview, Henry Farm, Oriole
34 Northwood Park, York University
35 East Toronto, Broadview North (Old East York)
36 Harbourfr

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Category,Venue,Venue Latitude,Venue Longitude
0,Parkwoods,43.75245,-79.32991,Fast Food Restaurant,KFC,43.754387,-79.333021
1,Parkwoods,43.75245,-79.32991,BBQ Joint,Bella Vita Catering & Private Chef Service,43.756651,-79.331524
2,Victoria Village,43.73057,-79.31306,Food Stand,Stellar's Lenonade Stand,43.731507,-79.311712
3,Victoria Village,43.73057,-79.31306,German Restaurant,Vinnia Meats,43.730465,-79.307520
4,"Regent Park, Harbourfront",43.65512,-79.36264,Breakfast Spot,Figs Breakfast & Lunch,43.655675,-79.364503


In [44]:
hotels=getNearbyPlaces(names=toronto['Neighborhood'],
                                            latitudes=toronto['Latitude'],
                                            longitudes=toronto['Longitude'] ,
                                            category=hotel_id )

hotels.head()

0 Parkwoods
1 Victoria Village
2 Regent Park, Harbourfront
3 Lawrence Manor, Lawrence Heights
4 Queen's Park, Ontario Provincial Government
5 Islington Avenue, Humber Valley Village
6 Malvern, Rouge
7 Don Mills
8 Parkview Hill, Woodbine Gardens
9 Garden District, Ryerson
10 Glencairn
11 West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
12 Rouge Hill, Port Union, Highland Creek
13 Don Mills
14 Woodbine Heights
15 St. James Town
16 Humewood-Cedarvale
17 Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
18 Guildwood, Morningside, West Hill
19 The Beaches
20 Berczy Park
21 Caledonia-Fairbanks
22 Woburn
23 Leaside
24 Central Bay Street
25 Christie
26 Cedarbrae
27 Hillcrest Village
28 Bathurst Manor, Wilson Heights, Downsview North
29 Thorncliffe Park
30 Richmond, Adelaide, King
31 Dufferin, Dovercourt Village
32 Scarborough Village
33 Fairview, Henry Farm, Oriole
34 Northwood Park, York University
35 East Toronto, Broadview North (Old East York)
36 Harbourfr

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Category,Venue,Venue Latitude,Venue Longitude
0,Parkwoods,43.75245,-79.32991,Hotel,Towns On The Ravine,43.754754,-79.332552
1,"Garden District, Ryerson",43.65739,-79.37804,Hotel,The Grand Hotel & Suites Toronto,43.656449,-79.374110
2,"Garden District, Ryerson",43.65739,-79.37804,Hotel,Marriott Downtown at CF Toronto Eaton Centre,43.654728,-79.382422
3,"Garden District, Ryerson",43.65739,-79.37804,Hotel,The Saint James Hotel,43.659398,-79.380932
4,"Garden District, Ryerson",43.65739,-79.37804,Hotel,Chelsea Hotel,43.658498,-79.383097


In [71]:
#Saving the first results in order to be able to use them in the future and since the api allowes a specified number of calls per day 
coffees.to_csv("coffees.csv")
restaurants.to_csv("restaurants.csv")
hotels.to_csv("hotels.csv")

In [62]:
#Since the results had a lot of venue categories I will rename them to coffee, so that the algorithm will label them as the same 
coffees.replace(['Café', 'Coffee Shop', 'Tea Room'],'Coffee Shop',inplace=True)
coffees.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Category,Venue,Venue Latitude,Venue Longitude
0,Parkwoods,43.75245,-79.32991,Coffee Shop,Tim Hortons,43.760668,-79.326368
2,Parkwoods,43.75245,-79.32991,Coffee Shop,La Notre,43.760704,-79.325396
3,Victoria Village,43.73057,-79.31306,Coffee Shop,Tim Hortons,43.725517,-79.313103
4,"Regent Park, Harbourfront",43.65512,-79.36264,Coffee Shop,Tandem Coffee,43.653559,-79.361809
5,"Regent Park, Harbourfront",43.65512,-79.36264,Coffee Shop,Rooster Coffee,43.651900,-79.365609


In [66]:
#Keeping only the restaurent label i.e. if the category is labeled as Chienese restaurant it will be restaurent 
def setting_all_venues_as_restaurant(x):
    the_chosen_word=str(x)
    return(the_chosen_word[-10:])

In [67]:
#since I was working on the project for more than a day I had to save and import the dataframes 
restaurants=pd.read_csv("restaurants.csv")
restaurants.drop("Unnamed: 0",axis=1,inplace=True)
restaurants.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Category,Venue,Venue Latitude,Venue Longitude
0,Parkwoods,43.75245,-79.32991,Fast Food Restaurant,KFC,43.754387,-79.333021
1,Victoria Village,43.73057,-79.31306,German Restaurant,Vinnia Meats,43.730465,-79.307520
2,"Regent Park, Harbourfront",43.65512,-79.36264,Italian Restaurant,Mangia and Bevi Resto-Bar,43.652250,-79.366355
3,"Regent Park, Harbourfront",43.65512,-79.36264,Thai Restaurant,Sukhothai,43.658444,-79.365681
4,"Regent Park, Harbourfront",43.65512,-79.36264,Restaurant,Impact Kitchen,43.656369,-79.356980


In [70]:
restaurants['Venue Category']=restaurants['Venue Category'].apply(setting_all_venues_as_restaurant)
restaurants['Venue Category'].shape

(1333,)

In [71]:
#The search query was for food I now am keeping only restaurants 
restaurants=restaurants[restaurants['Venue Category']=='Restaurant']
restaurants.shape

(976, 7)

In [74]:
hotels=pd.read_csv("hotels.csv")
hotels.drop("Unnamed: 0",axis=1,inplace=True)
hotels.shape

(274, 7)

In [75]:
#Keeping only the hotels 
hotels=hotels[hotels['Venue Category']=='Hotel']
hotels.shape

(201, 7)

In [76]:
#Removing Duplicate rows 
print("Hotels before removing",hotels.shape)
print("Resturants before removing",restaurants.shape)
print("Coffees before removing",coffees.shape)
hotels.drop_duplicates(subset=['Venue','Neighborhood'],inplace=True)
restaurants.drop_duplicates(subset=['Venue','Neighborhood'],inplace=True)
coffees.drop_duplicates(subset=['Venue','Neighborhood'],inplace=True)
print("Hotels after removing",hotels.shape)
print("Resturants after removing",restaurants.shape)
print("Coffees after removing",coffees.shape)


Hotels before removing (201, 7)
Resturants before removing (976, 7)
Coffees before removing (1372, 7)
Hotels after removing (201, 7)
Resturants after removing (962, 7)
Coffees after removing (1372, 7)


In [77]:
#One last save 
hotels.to_csv("updated hotels.csv",index=False)
restaurants.to_csv("updated restaurants.csv",index=False)
coffees.to_csv("updated coffee shops.csv",index=False)

In [78]:
places=[hotels,restaurants,coffees]
final_df=pd.concat(places)
final_df.to_csv("Final data.csv",index=False)

In [6]:
final_df=pd.read_csv("Final data.csv")

In [7]:
final_df.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Category,Venue,Venue Latitude,Venue Longitude
0,Parkwoods,43.75245,-79.32991,Hotel,Towns On The Ravine,43.754754,-79.332552
1,"Garden District, Ryerson",43.65739,-79.37804,Hotel,The Grand Hotel & Suites Toronto,43.656449,-79.374110
2,"Garden District, Ryerson",43.65739,-79.37804,Hotel,Marriott Downtown at CF Toronto Eaton Centre,43.654728,-79.382422
3,"Garden District, Ryerson",43.65739,-79.37804,Hotel,The Saint James Hotel,43.659398,-79.380932
4,"Garden District, Ryerson",43.65739,-79.37804,Hotel,Chelsea Hotel,43.658498,-79.383097


In [8]:
toronto_onehot = pd.get_dummies(final_df[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = final_df['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Coffee Shop,Hotel,Restaurant
0,Parkwoods,0,1,0
1,"Garden District, Ryerson",0,1,0
2,"Garden District, Ryerson",0,1,0
3,"Garden District, Ryerson",0,1,0
4,"Garden District, Ryerson",0,1,0


In [36]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').sum().reset_index()
toronto_grouped

,Neighborhood,Coffee Shop,Hotel,Restaurant
0,Agincourt,4,0,6
1,"Alderwood, Long Branch",3,0,3
2,"Bathurst Manor, Wilson Heights, Downsview North",2,0,0
3,Bayview Village,1,0,0
4,"Bedford Park, Lawrence Manor East",3,0,12
...,...,...,...,...
92,"Willowdale, Willowdale East",14,1,6
93,"Willowdale, Willowdale West",1,0,0
94,Woburn,2,0,3
95,Woodbine Heights,9,0,2


In [50]:
#Adding the latitude and longitude to the data frame because they are imortant features 
ml_df=toronto_grouped.set_index('Neighborhood').join(toronto.set_index('Neighborhood'),how='left')
ml_df.reset_index(inplace=True)
ml_df

,Neighborhood,Coffee Shop,Hotel,Restaurant,Postal Code,Borough,Latitude,Longitude
0,Agincourt,4,0,6,M1S,Scarborough,43.79452,-79.26708
1,"Alderwood, Long Branch",3,0,3,M8W,Etobicoke,43.60124,-79.53879
2,"Bathurst Manor, Wilson Heights, Downsview North",2,0,0,M3H,North York,43.75788,-79.44847
3,Bayview Village,1,0,0,M2K,North York,43.78112,-79.38060
4,"Bedford Park, Lawrence Manor East",3,0,12,M5M,North York,43.73545,-79.41916
...,...,...,...,...,...,...,...,...
96,"Willowdale, Willowdale East",14,1,6,M2N,North York,43.76774,-79.40728
97,"Willowdale, Willowdale West",1,0,0,M2R,North York,43.77989,-79.44678
98,Woburn,2,0,3,M1G,Scarborough,43.76812,-79.21761
99,Woodbine Heights,9,0,2,M4C,East York,43.68970,-79.30682


K-means algorithm
---------------------------
since the data has been cleaned I now can use the algorithm to obtain the places where the venues could be successful 

In [105]:
kclusters = 3
toronto_grouped_clustering=ml_df.drop(['Neighborhood','Postal Code','Borough'], 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
#Adding the cluster labels to the data frame
toronto_grouped_clustering['Cluster Labels']=kmeans.labels_
#Adding the Neighborhoods and Boroughs to the data frame 
toronto_grouped_clustering['Neighborhood']=ml_df['Neighborhood']
toronto_grouped_clustering['Borough']=ml_df['Borough']
toronto_grouped_clustering.rename(columns={"Coffee Shop":"Coffee Shop Count","Restaurant":"Restaurant Count","Hotel":"Hotel Count"},inplace=True)
toronto_grouped_clustering.head()

,Coffee Shop Count,Hotel Count,Restaurant Count,Latitude,Longitude,Cluster Labels,Neighborhood,Borough
0,4,0,6,43.79452,-79.26708,0,Agincourt,Scarborough
1,3,0,3,43.60124,-79.53879,0,"Alderwood, Long Branch",Etobicoke
2,2,0,0,43.75788,-79.44847,0,"Bathurst Manor, Wilson Heights, Downsview North",North York
3,1,0,0,43.78112,-79.38060,0,Bayview Village,North York
4,3,0,12,43.73545,-79.41916,0,"Bedford Park, Lawrence Manor East",North York


In [115]:
toronto_grouped_clustering.to_csv("Toronto with Cluster.csv",index=False)

In [106]:
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
latitude=43.6532
longitude=-79.3832

toronto_clustered=folium.Map(location=[latitude,longitude],zoom_start=10)
# add markers to the map
markers_colors = []
for lat, lon, neigh, bor,cluster in zip(toronto['Latitude'], toronto_grouped_clustering['Longitude'], toronto_grouped_clustering['Neighborhood'], toronto_grouped_clustering['Borough'],toronto_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(neigh) + ' ,' + str(bor)+" With Label"+str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(toronto_clustered)
toronto_clustered

In [107]:
toronto_clustered.save("toronto clustered.html")

Now let's examine each cluster
=========================

First Cluster
----------------

In [111]:
toronto_grouped_clustering.loc[toronto_grouped_clustering['Cluster Labels'] == 0, toronto_grouped_clustering.columns[ list(range(0,toronto_grouped_clustering.shape[1]))]]

,Coffee Shop Count,Hotel Count,Restaurant Count,Latitude,Longitude,Cluster Labels,Neighborhood,Borough
0,4,0,6,43.79452,-79.26708,0,Agincourt,Scarborough
1,3,0,3,43.60124,-79.53879,0,"Alderwood, Long Branch",Etobicoke
2,2,0,0,43.75788,-79.44847,0,"Bathurst Manor, Wilson Heights, Downsview North",North York
3,1,0,0,43.78112,-79.38060,0,Bayview Village,North York
4,3,0,12,43.73545,-79.41916,0,"Bedford Park, Lawrence Manor East",North York
...,...,...,...,...,...,...,...,...
96,14,1,6,43.76774,-79.40728,0,"Willowdale, Willowdale East",North York
97,1,0,0,43.77989,-79.44678,0,"Willowdale, Willowdale West",North York
98,2,0,3,43.76812,-79.21761,0,Woburn,Scarborough
99,9,0,2,43.68970,-79.30682,0,Woodbine Heights,East York


Second Cluster
---------------------

In [112]:
toronto_grouped_clustering.loc[toronto_grouped_clustering['Cluster Labels'] == 1, toronto_grouped_clustering.columns[ list(range(0,toronto_grouped_clustering.shape[1]))]]

,Coffee Shop Count,Hotel Count,Restaurant Count,Latitude,Longitude,Cluster Labels,Neighborhood,Borough
8,46,17,56,43.64869,-79.38544,1,"Business reply mail Processing Centre, South C...",East Toronto
11,46,17,56,43.64869,-79.38544,1,Canada Post Gateway Processing Centre,Mississauga
18,43,19,51,43.64840,-79.37914,1,"Commerce Court, Victoria Hotel",Downtown Toronto
33,43,21,47,43.64828,-79.38146,1,"First Canadian Place, Underground city",Downtown Toronto
35,53,12,50,43.65739,-79.37804,1,"Garden District, Ryerson",Downtown Toronto
69,45,21,49,43.64970,-79.38258,1,"Richmond, Adelaide, King",Downtown Toronto
80,46,17,56,43.64869,-79.38544,1,Stn A PO Boxes,Downtown Toronto
88,41,23,46,43.64710,-79.38153,1,"Toronto Dominion Centre, Design Exchange",Downtown Toronto


Third Cluster
=========

In [114]:
toronto_grouped_clustering.loc[toronto_grouped_clustering['Cluster Labels'] == 2, toronto_grouped_clustering.columns[ list(range(0,toronto_grouped_clustering.shape[1]))]]

,Coffee Shop Count,Hotel Count,Restaurant Count,Latitude,Longitude,Cluster Labels,Neighborhood,Borough
5,39,4,31,43.64536,-79.37306,2,Berczy Park,Downtown Toronto
7,29,1,24,43.63941,-79.42676,2,"Brockton, Parkdale Village, Exhibition Place",West Toronto
9,41,2,30,43.64082,-79.39818,2,"CN Tower, King and Spadina, Railway Lands, Har...",Downtown Toronto
13,56,7,26,43.65609,-79.38493,2,Central Bay Street,Downtown Toronto
15,54,6,35,43.66659,-79.38133,2,Church and Wellesley,Downtown Toronto
39,40,10,21,43.64285,-79.38076,2,"Harbourfront East, Union Station, Toronto Islands",Downtown Toronto
48,74,2,24,43.65351,-79.39722,2,"Kensington Market, Chinatown, Grange Park",Downtown Toronto
53,47,0,26,43.64848,-79.41774,2,"Little Portugal, Trinity",West Toronto
67,68,0,9,43.66253,-79.39188,2,"Queen's Park, Ontario Provincial Government",Downtown Toronto
68,32,0,7,43.65512,-79.36264,2,"Regent Park, Harbourfront",Downtown Toronto


Results
=====

Refering to the above data frames it can be found that : 
the first cluster has relatively low coffee shops and restaurants , and almost no hotels 
the second cluster ,however, has the highest count of any cluster 
the third cluster is in the middle

Conclusion
=========

Having checeked the data we can presume that the best places to open the venues are in cluster 2 for hotels since it has the most count ,the third for restaurants and coffees can be opened anywhere 